[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/emcarthur/WeeklyPDFPlannerCreator/blob/main/WeeklyPlanner.ipynb)



## Config: set these to customize calendar

In [57]:
# Set year
YEAR = 2022

# This part depends on how your template is set up. With my current template 
# only Monday works accurately without edits.
WEEK_START_DAY = 'Monday' # 'Sunday'

# Name output
OUTFILE_NAME = f'{YEAR}_WeeklyPlanner.pdf'


## Install and load dependencies

In [43]:
!pip3 install cairosvg

     |████████████████████████████████| 45 kB 2.4 MB/s 
     |████████████████████████████████| 88 kB 4.5 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.whl size=89667 sha256=f9ca14a788e6ffe02b8c8bcf84735b9b29b1d0930acf431ac696d3cbc0348328
  Stored in directory: /root/.cache/pip/wheels/4e/ca/e1/5c8a9692a27f639a07c949044bec943f26c81cd53d3805319f
Successfully built cairocffi


In [44]:
import cairosvg
import tempfile
import numpy as np
import copy
import time
from datetime import timedelta, datetime

If loaded packages aren't working, directly install dependencies from requirements.txt



```
## How I exported requirements (don't redo):
#!pip3 install cairosvg
#!pip freeze > requirements.txt 

# Import requirements:
!pip install -r requirements.txt
```



## Create and load in template


*   *Optional:* I first drafted out a template using MS word with some tables: `template_draft.docx`. I then exported this as a pdf: `template_draft.pdf`. 
*   Then I imported the pdf into [inkscape (free)](https://inkscape.org/) to save an editable svg. You can directly just create a template in inkscape without MS word.
*   I edited the svg to look exactly how I wanted. Make sure to set the document properties size to the size of the document you want (e.g. A5, A6).
*   I created some placeholder text for parts of the template that I wanted to be dynamically updated (eg. "Month1 Day1" or "D1" "D2"). Just make sure this text is centered and where you want the final text to end up.
*   *Optional:* I converted other text to objects so their formatting is always consistent (Path > Object to Path)
*   Save this as `template.svg`



In [53]:
template = []
with open('template.svg') as f:
    template = f.readlines()


## Export planner

### Helper functions

In [54]:
def updateSVGdates(template, date):
  # Reads in template and first date of the week, edits svg template by replacing
  # placeholder text ("Month1 Day2 - Month2 Day2", "D#")
  template_edit = copy.deepcopy(template)
  for i, l in enumerate(template_edit):
    if "Month1 Day2 - Month2 Day2" in l:
      template_edit[i] = l.replace("Month1 Day2 - Month2 Day2", f"{date.strftime('%b')} {date.strftime('%d').lstrip('0')} - {(date + timedelta(days=6)).strftime('%b')} {(date + timedelta(days=6)).strftime('%d').lstrip('0')}")
    elif "D1" in l:
      template_edit[i] = l.replace("D1",date.strftime("%d").lstrip('0'))
    elif "D2" in l:
      template_edit[i] = l.replace("D2",(date + timedelta(days=1)).strftime("%d").lstrip('0'))
    elif "D3" in l:
      template_edit[i] = l.replace("D3",(date + timedelta(days=2)).strftime("%d").lstrip('0'))
    elif "D4" in l:
      template_edit[i] = l.replace("D4",(date + timedelta(days=3)).strftime("%d").lstrip('0'))
    elif "D5" in l:
      template_edit[i] = l.replace("D5",(date + timedelta(days=4)).strftime("%d").lstrip('0'))
    elif "D6" in l:
      template_edit[i] = l.replace("D6",(date + timedelta(days=5)).strftime("%d").lstrip('0'))
    elif "D7" in l:
      template_edit[i] = l.replace("D7",(date + timedelta(days=6)).strftime("%d").lstrip('0'))
  return template_edit

def saveSingleSVG(lines,dir,num):
  # Write list to svg
  out = open(f"{dir}/{num}.svg", "w")
  for l in lines:
      out.write(l + "\n")
  out.close()

In [55]:
# Combine svgs into one pdf
# https://github.com/Kozea/CairoSVG/issues/200
import cairocffi
from cairosvg.parser import Tree
from cairosvg.surface import PDFSurface

class RecordingPDFSurface(PDFSurface):
    surface_class = cairocffi.RecordingSurface

    def _create_surface(self, width, height):
        cairo_surface = cairocffi.RecordingSurface(cairocffi.CONTENT_COLOR_ALPHA, (0, 0, width, height))
        return cairo_surface, width, height  

def convert_list(urls, write_to, dpi=72):
    surface = cairocffi.PDFSurface(write_to, 1, 1)
    context = cairocffi.Context(surface)
    for url in urls:
        image_surface = RecordingPDFSurface(Tree(url=url), None, dpi)
        surface.set_size(image_surface.width, image_surface.height)
        context.set_source_surface(image_surface.cairo, 0, 0)
        context.paint()
        surface.show_page()
    surface.finish()

## Create planner
Takes about 20-30 seconds

In [58]:
# https://stackoverflow.com/questions/2003870/how-can-i-select-all-of-the-sundays-for-a-year-using-python

# Determine all Mondays with weeks that overlap the year specified
first_date = f'{YEAR}-01-01'
final_date = f'{YEAR}-12-31'
first_date = datetime.strptime(first_date, '%Y-%m-%d') - timedelta(days=6)
last_date = datetime.strptime(final_date, '%Y-%m-%d')
week_day = WEEK_START_DAY
dates = [first_date + timedelta(days=x) for x in range((last_date - first_date).days + 1) if (first_date + timedelta(days=x)).weekday() == time.strptime(week_day, '%A').tm_wday]

print(f'Number of weeks: {len(dates)}')

# Edit the template for each week, save a an tmp svg, combine all tmp svgs into one
with tempfile.TemporaryDirectory() as tmpdirname:
  for i,date in enumerate(dates):
    edited = updateSVGdates(template,date)
    saveSingleSVG(edited,tmpdirname,i)
  
  convert_list([f"{tmpdirname}/{x}.svg" for x in range(len(dates))], OUTFILE_NAME)

print("Done!")


Number of weeks: 53
Done!


## Download your planner & add bookmarks


*   If you want to add monthly bookmarks to your pdf, you can use [PDF Escape](https://www.pdfescape.com/) (free), Adobe acrobat (not free), or [other online tools](https://www.ilovefreesoftware.com/21/featured/online-add-bookmarks-to-pdf-free-websites.html).
*   With PDF escape, upload your pdf. Go to the "Bookmarks" panel (looks like a page with a little red tab). Scroll to the page you want to add a bookmark to. Click "add" and title your bookmark. (Tip: use the view:fit to precisely bookmark the right page).



## Behind the scenes
To mount to Google Drive & push to GitHub



```
from google.colab import drive

ROOT = '/content/drive'
PROJ = f'{ROOT}/MyDrive/Projects/WeeklyPDFPlannerCreator'
GIT_USERNAME = "emcarthur"
GIT_TOKEN = "********"
GIT_REPOSITORY = "WeeklyPDFPlannerCreator"

drive.mount(ROOT) # mount google drive files

%cd {PROJ}
!git init # initialize repo
!git add .
!git config user.email "evonne.mcarthur@gmail.com"
!git config user.name "emcarthur"
!git commit -m "add initial documents"
!git branch -M main
GIT_PATH = f"https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
!git remote add origin {GIT_PATH}
!git push -u origin main

# If token changes
GIT_TOKEN = "********"
GIT_PATH = f"https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
!git remote set-url origin {GIT_PATH}

```



In [79]:
!ls

2021_WeeklyPlanner.pdf		      template_draft.docx
2021_WeeklyPlanner_withBookmarks.pdf  template_draft.pdf
2022_WeeklyPlanner.pdf		      template.svg
2022_WeeklyPlanner_withBookmarks.pdf  WeeklyPlanner.ipynb
requirements.txt


In [76]:
!git push -u origin main

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (4/4), 351 bytes | 70.00 KiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/emcarthur/WeeklyPDFPlannerCreator.git
   019afb9..81cc459  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
